In [12]:
# docker file used for this notebook for pytorch 2.0
# docker pull cnstark/pytorch:2.0.1-py3.10.11-cuda11.8.0-ubuntu22.04


# Семинар torch.compile 

Cеминар основан на документации [1] и ноутбуке [2].<br>

[1] https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html <br>
[2] https://colab.research.google.com/github/PyTorchKorea/tutorials-kr/blob/master/docs/_downloads/96ad88eb476f41a5403dcdade086afb8/torch_compile_tutorial.ipynb <br>

``torch.compile`` новый компилятор для ускорения кода на PyTorch. <br>
Это JIT-компилятор с использованием оптимизированных кернелей,
который требует минимального изменения кода. <br>

В этом семинаре мы рассмотрим базовое использование ``torch.compile``, а так же сравним его с [TorchScript](https://pytorch.org/docs/stable/jit.html) и
[FX Tracing](https://pytorch.org/docs/stable/fx.html#torch.fx.symbolic_trace).

**Содержание**

- Базовый пример
- Ускорение с помощью ``torch.compile``
- Сравнение с TorchScript и FX Tracing
- Заключение

**Required pip Dependencies**

- ``torch >= 2.0``
- ``torchvision``
- ``numpy``
- ``scipy``
- ``tabulate``

docker file used for this notebook for pytorch 2.0: <br>
``docker pull cnstark/pytorch:2.0.1-py3.10.11-cuda11.8.0-ubuntu22.04``





ПРИМЕЧАНИЕ. Результаты зависят от версии GPU, что бы воспроизвести результаты команда ``torch.compile`` рекомендует использование NVIDIA (H100, A100 или V100). 

In [1]:
import numpy as np

import torch
import warnings

gpu_ok = False
if torch.cuda.is_available():
    device_cap = torch.cuda.get_device_capability()
    if device_cap in ((7, 0), (8, 0), (9, 0)):
        
        gpu_ok = True

if not gpu_ok:
    warnings.warn(
        "GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower "
        "than expected."
    )

/tmp/ipykernel_72586/146755034.py:14: UserWarning: GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
  warnings.warn(


## Базовый пример

Произвольные функции Python можно оптимизировать, передав вызываемый объект в
``torch.compile``. Затем мы можем вызвать возвращенную оптимизированную
функцию вместо исходной.



In [11]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b

opt_foo = torch.compile(foo,  mode="reduce-overhead")

print(opt_foo(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 1.9886e+00,  4.1127e-01,  1.4072e+00,  1.2928e+00,  1.5151e+00,
          2.9439e-01, -4.2560e-01,  2.3742e-01,  1.7742e+00,  1.8396e+00],
        [ 1.5106e+00, -5.8609e-01,  8.2860e-02,  8.3027e-01,  3.4428e-01,
         -1.0144e-01,  1.9328e-01,  7.4056e-01,  9.1863e-01,  1.6308e+00],
        [-1.5548e-01,  1.3780e-01, -3.5028e-01,  1.5946e-01,  9.4538e-01,
          1.3192e+00, -3.0626e-01,  1.7055e+00,  3.2916e-01,  7.0887e-01],
        [-1.5760e+00,  1.9290e+00,  1.4237e+00, -5.0070e-01,  8.6380e-01,
         -9.4732e-01,  4.2540e-01, -9.2554e-01,  1.6768e+00,  5.7387e-01],
        [ 7.1218e-02, -3.4733e-02,  2.1215e-01,  1.0887e+00, -8.2437e-03,
          1.2018e+00, -9.4699e-02,  3.7755e-02, -7.7347e-01,  1.3405e+00],
        [ 1.7557e+00,  9.6908e-03,  1.1708e+00,  1.7649e+00,  4.1224e-01,
          1.1330e-01, -2.7980e-01,  1.6276e+00, -1.6754e-01,  5.3518e-01],
        [ 4.6720e-01,  1.8873e+00,  1.7394e+00,  9.2679e-01, -5.5992e-01,
         -1.9202e-02,  7.2074e-0

Так же можно использовать декоратор

In [12]:
@torch.compile
def opt_foo2(x, y):
    a = torch.sin(x)
    b = torch.cos(x)
    return a + b
print(opt_foo2(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 1.4117,  0.5819,  0.6219,  0.4882,  1.4083,  0.8319, -0.9973,  1.0937,
          1.3785, -0.1518],
        [ 1.1816,  0.2198,  0.6366, -1.1603,  0.3358,  1.4087,  0.6863,  1.3609,
          0.8532,  0.7937],
        [ 0.3308, -1.1071,  1.3676, -0.0661,  1.2262, -1.2424,  1.3208,  1.4070,
         -1.3659,  1.0636],
        [-1.0567,  1.4129, -0.3669,  1.1357, -0.3752, -1.2310,  1.1175,  0.6678,
          1.3118, -0.5076],
        [-0.0456, -0.4552,  0.2228, -0.5175,  1.3639,  0.8457,  1.2857,  1.3490,
          0.5475,  1.3128],
        [ 0.0852,  0.7674, -0.9651,  0.9469,  1.2172, -0.3725, -0.6997, -1.2694,
          1.3835,  1.3082],
        [ 1.0041,  0.9339,  1.2820,  1.1545, -0.3292,  1.4131, -1.3757,  0.9799,
         -0.0876, -1.3564],
        [-0.3614,  0.6711,  0.7682,  1.2020, -0.3152,  0.6595,  0.1493, -0.1035,
          0.9917,  0.8256],
        [ 0.9233, -1.1307,  1.0705,  0.6847,  0.6727, -0.3738,  0.3275,  1.3669,
         -0.5436, -0.3676],
        [ 1.3225, -

/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:279: UserWarning: changing options to `torch.compile()` may require calling `torch._dynamo.reset()` to take effect
  warnings.warn(


Мы можем оптимизировать целый модуль ``torch.nn.Module``.



In [13]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))

mod = MyModule()
opt_mod = torch.compile(mod)
print(opt_mod(torch.randn(10, 100)))

tensor([[0.0000, 0.5243, 0.6419, 0.4991, 1.1962, 1.0545, 0.0000, 0.3147, 0.0000,
         0.0982],
        [0.0000, 0.0000, 0.0000, 0.2249, 0.2523, 0.0857, 0.2069, 0.4119, 0.3501,
         0.0000],
        [0.1905, 0.5348, 0.0000, 0.0000, 0.2223, 1.0515, 0.0000, 0.0000, 0.0000,
         0.6836],
        [0.0000, 0.4371, 0.3040, 0.3571, 0.0000, 0.0356, 0.0000, 0.3245, 0.0356,
         0.0000],
        [0.8753, 0.0000, 0.0000, 0.0000, 0.0123, 0.2211, 0.2291, 0.0000, 0.6873,
         0.1751],
        [0.1301, 0.2727, 0.0000, 0.0000, 0.3780, 0.6455, 0.0286, 0.0000, 0.0000,
         1.1182],
        [0.0000, 0.7593, 0.0625, 0.0000, 0.1683, 0.0000, 1.2396, 0.5417, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1221, 0.0000, 0.0000, 0.0784, 0.0707, 0.5859, 0.3438,
         0.0284],
        [0.0000, 0.2741, 0.0000, 0.0000, 0.3410, 0.0000, 0.7112, 0.0000, 0.0000,
         0.6204],
        [0.0431, 0.0000, 0.0000, 0.0000, 0.0000, 0.3431, -0.0000, 0.0024, -0.0000,
         -0.0000]], grad_f


## Ускорение для инференса

Давайте теперь посмтрим, как использование ``torch.compile`` может ускорить <br>
модели. Мы сравним стандартный eager режим и ``torch.compile`` для инфиренса и для обучения ResNet-18 на случайных данных.

Но прежде чем мы начнем, нам нужно определить некоторые служебные функции.


In [5]:
# Returns the result of running `fn()` and the time it took for `fn()` to run,
# in seconds. We use CUDA events and synchronization for the most accurate
# measurements.
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

# Generates random input and targets data for the model, where `b` is
# batch size.
def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )

from torchvision.models import densenet121
def init_model():
    return densenet121().to(torch.float32).cuda()

Первым делом мы сравним инфиренс, но прежде посмотрим сколько времени занимает компиляция.

В  ``torch.compile`` мы используем некотрые параметры для переменной ``mode``, мы обсудим их позже.


In [14]:
def evaluate(mod, inp):
    return mod(inp)

model = init_model()

# Reset since we are using a different mode.
import torch._dynamo
torch._dynamo.reset()

evaluate_opt = torch.compile(evaluate, mode="reduce-overhead")

inp = generate_data(16)[0]

# One Iteration for each function 
print("eager:", timed(lambda: evaluate(model, inp))[1])
print("compile:", timed(lambda: evaluate_opt(model, inp))[1])

eager: 0.029458431243896483
compile: 59.71681640625


Обратите внимание, что выполнение ``torch.compile`` занимает намного больше времени <br>
по сравнению с ``eager mode``. Это потому, что ``torch.compile`` занимается компиляцией и  <br>
оптимизирует ядра по мере ее выполнения.  Здесь мы запускаем нашу модель один раз. <br>

Если мы будем запускать нашу модель несколько раз, то после компиляции, которая происходит на первом этапе мы увидим ускорение. <br>

In [15]:
N_ITERS = 10


eager_times = []
compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    _, eager_time = timed(lambda: evaluate(model, inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    _, compile_time = timed(lambda: evaluate_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

import numpy as np
eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 0.05177231979370117
eager eval time 1: 0.04363151931762695
eager eval time 2: 0.03373040008544922
eager eval time 3: 0.03276486587524414
eager eval time 4: 0.036166431427001954
eager eval time 5: 0.03266156768798828
eager eval time 6: 0.02955078315734863
eager eval time 7: 0.027540735244750977
eager eval time 8: 0.036450302124023434
eager eval time 9: 0.027553152084350586
~~~~~~~~~~
compile eval time 0: 0.018311391830444335
compile eval time 1: 0.016648191452026367
compile eval time 2: 0.01970390319824219
compile eval time 3: 0.018268159866333008
compile eval time 4: 0.01830441665649414
compile eval time 5: 0.018327520370483397
compile eval time 6: 0.018128671646118165
compile eval time 7: 0.01819647979736328
compile eval time 8: 0.018149471282958983
compile eval time 9: 0.011595680236816406
~~~~~~~~~~
(eval) eager median: 0.03324763298034668, compile median: 0.018232319831848143, speedup: 1.8235547251792856x
~~~~~~~~~~


И действительно, мы видим, что запуск нашей модели с помощью ``torch.compile``
приводит к значительному ускорению. Ускорение в основном достигается за счет сокращения накладных расходов Python и
Чтение/запись графического процессора, поэтому наблюдаемое ускорение может варьироваться в зависимости от таких факторов, как 
архитектура модели и ее размер. <br>

<br>
Например, если архитектура модели простя, 
но занимает много памяти, то узким местом будет 
вычисления графического процессора и наблюдаемое ускорение может быть менее значительными. <br>


``torch.compile`` поддерживает разные режими компиляции. Подробнее про разные режимы можно посмотреть тут: [here](https://pytorch.org/get-started/pytorch-2.0/#user-experience).


```python
# default: optimizes for large models, low compile-time
#          and no extra memory usage
torch.compile(model)

# reduce-overhead: optimizes to reduce the framework overhead
#                and uses some extra memory. Helps speed up small models
torch.compile(model, mode="reduce-overhead")

# max-autotune: optimizes to produce the fastest model,
#               but takes a very long time to compile
torch.compile(model, mode="max-autotune")
```



В целом, для тестирования моделей PyTorch лучше использовать ``torch.utils.benchmark`` вместо самопальной timed. <br>

<br>

В этом примере нам нужна была такая функция что бы показать задержки на компиляцию.

<br>
<br>

**Посмотрим на сравнение скорости обучения.**


In [16]:
N_ITERS = 10

model = init_model()
opt = torch.optim.Adam(model.parameters())

def train(mod, data):
    opt.zero_grad(True)
    pred = mod(data[0])
    loss = torch.nn.CrossEntropyLoss()(pred, data[1])
    loss.backward()
    opt.step()

eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, eager_time = timed(lambda: train(model, inp))
    eager_times.append(eager_time)
    print(f"eager train time {i}: {eager_time}")
print("~" * 10)

model = init_model()
opt = torch.optim.Adam(model.parameters())
train_opt = torch.compile(train, mode="reduce-overhead")

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager train time 0: 0.23956906127929686
eager train time 1: 0.06768262481689453
eager train time 2: 0.06373747253417969
eager train time 3: 0.07324774169921874
eager train time 4: 0.07006269073486328
eager train time 5: 0.07524041748046875
eager train time 6: 0.07436246490478515
eager train time 7: 0.06958281707763672
eager train time 8: 0.06574285125732422
eager train time 9: 0.06812441253662109
~~~~~~~~~~
compile train time 0: 87.491828125
compile train time 1: 0.04038835144042969
compile train time 2: 0.031593280792236327
compile train time 3: 0.03105996894836426
compile train time 4: 0.0311648006439209
compile train time 5: 0.033407169342041014
compile train time 6: 0.0331358413696289
compile train time 7: 0.03295011138916016
compile train time 8: 0.03601126480102539
compile train time 9: 0.03607596969604492
~~~~~~~~~~
(train) eager median: 0.06982275390625, compile median: 0.03327150535583496, speedup: 2.098575136877746x
~~~~~~~~~~


Опять, мы видим, что ``torch.compile`` в первом случае занимает больше времени.
итерации, так как она должна скомпилировать модель, но на последующих итерациях мы видим
значительное ускорение по сравнению с нетерпеливым.


## Cравнение с TorchScript и FX Tracing

Мы видели что ``torch.compile`` может ускорить вычисления. <br>

Но все равно, почему следует использовать  ``torch.compile`` вместо ``TorchScript`` или ``FX Tracing``? <br>


Прежде всего, преимущество ``torch.compile`` заключается в его способности работать
почти с произвольным кодом на Python с его минимальными изменениями. <br>

А так же ``torch.compile`` может работать с data-dependent control flow ``if x.sum() < 0:``.



In [22]:
def f1(x, y):
    if x.sum() < 0:
        return -y
    return y

# Test that `fn1` and `fn2` return the same result, given
# the same arguments `args`. Typically, `fn1` will be an eager function
# while `fn2` will be a compiled function (torch.compile, TorchScript, or FX graph).
def test_fns(fn1, fn2, args):
    out1 = fn1(*args)
    out2 = fn2(*args)
    return torch.allclose(out1, out2)

inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)

TorchScript tracing ``f1`` привдет к неправильным результатам, так зафиксируется только один проход по данным.



In [23]:
traced_f1 = torch.jit.trace(f1, (inp1, inp2))
print("traced 1, 1:", test_fns(f1, traced_f1, (inp1, inp2)))
print("traced 1, 2:", test_fns(f1, traced_f1, (-inp1, inp2)))

traced 1, 1: True
traced 1, 2: False


/tmp/ipykernel_72586/4005623017.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:


```
TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:
```

FX tracing ``f1`` приводит к ошибке из-за присутствия поток управления, зависящий от данных.



In [24]:
import traceback as tb
try:
    torch.fx.symbolic_trace(f1)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_72586/1451191637.py", line 3, in <module>
    torch.fx.symbolic_trace(f1)
  File "/usr/local/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 1109, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/fx/_symbolic_trace.py", line 778, in trace
    (self.create_arg(fn(*args)),),
  File "/tmp/ipykernel_72586/4005623017.py", line 2, in f1
    if x.sum() < 0:
  File "/usr/local/lib/python3.10/site-packages/torch/fx/proxy.py", line 413, in __bool__
    return self.tracer.to_bool(self)
  File "/usr/local/lib/python3.10/site-packages/torch/fx/proxy.py", line 276, in to_bool
    raise TraceError('symbolically traced variables cannot be used as inputs to control flow')
torch.fx.proxy.TraceError: symbolically traced variables cannot be 

Если мы предоставим значение для ``x`` при попытке  FX  trace ``f1``, тогда
мы сталкиваемся с той же проблемой, что и при TorchScript, из за зависимости графа от данных.

In [26]:
fx_f1 = torch.fx.symbolic_trace(f1, concrete_args={"x": inp1})
print("fx 1, 1:", test_fns(f1, fx_f1, (inp1, inp2)))
print("fx 1, 2:", test_fns(f1, fx_f1, (-inp1, inp2)))

fx 1, 1: True
fx 1, 2: False


В это время ``torch.compile`` корректно обрабатывает динамический control-flow.



In [27]:
# Reset since we are using a different mode.
torch._dynamo.reset()

compile_f1 = torch.compile(f1)
print("compile 1, 1:", test_fns(f1, compile_f1, (inp1, inp2)))
print("compile 1, 2:", test_fns(f1, compile_f1, (-inp1, inp2)))
print("~" * 10)

compile 1, 1: True
compile 1, 2: True
~~~~~~~~~~


TorchScript scripting может работать в этой ситуации, но нам нужно будет адаптировать код и убедиться что у нас используется статическое типизирование данных.



In [31]:
def f2(x, y):
    return x + y

inp1 = torch.randn(5, 5)
inp2 = 3

script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Другое преимущество ``torch.compile``  в сравнении с 
предидущими решениями это возможность использлвать функции из PyTorch.



In [32]:
import scipy

def f3(x):
    x = x * 2
    x = scipy.fft.dct(x.numpy())
    x = torch.from_numpy(x)
    x = x * 2
    return x

TorchScript обрабатывает результаты вызовов функций, отличных от PyTorch.
как константы, и поэтому результаты могут быть ошибочными.

In [33]:
inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)
traced_f3 = torch.jit.trace(f3, (inp1,))
print("traced 3:", test_fns(f3, traced_f3, (inp2,)))

traced 3: False


/tmp/ipykernel_72586/2558593883.py:5: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = scipy.fft.dct(x.numpy())
/tmp/ipykernel_72586/2558593883.py:6: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)


``torch.compile`` 


In [34]:
compile_f3 = torch.compile(f3)
print("compile 3:", test_fns(f3, compile_f3, (inp2,)))

compile 3: True


## TorchDynamo и FX Graphs

Одна из важных компонент ``torch.compile`` это TorchDynamo.
TorchDynamo отвечает за JIT компиляцию кода на Python в
[FX graphs](https://pytorch.org/docs/stable/fx.html#torch.fx.Graph), который далее может быть оптимизирован. TorchDynamo выделяет FX графы анализруя Питоновский байткод во время выполнения, а так же отслеживает вызовы к функциям PyTorch.

TorchInductor, другая компонента ``torch.compile``,
конвертирует FX граф в оптимизированные кернели, а
TorchDynamo позволяет использовать различные бекнеды.  <br>


Давайте создадим свой бекенд, который выводит FX граф и не оптимизированный forward (что-то типа принта, но вызывывается он TorchDynamo). 

In [36]:
from typing import List
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

# Reset since we are using a different backend.
torch._dynamo.reset()

opt_model = torch.compile(init_model(), backend=custom_backend)
opt_model(generate_data(16)[0])

custom backend called with FX graph:
opcode         name                                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

tensor([[ 0.1929,  0.0299,  0.0168,  ...,  0.4473, -0.2145,  0.0311],
        [ 0.1494,  0.0819, -0.1082,  ...,  0.3111, -0.1846,  0.2114],
        [ 0.2941, -0.1400, -0.2095,  ...,  0.1943, -0.1441,  0.2407],
        ...,
        [ 0.1574, -0.1153, -0.1263,  ...,  0.3258, -0.0752,  0.2275],
        [ 0.1529, -0.0828, -0.1764,  ...,  0.3981, -0.0165,  0.2563],
        [ 0.2552, -0.0129, -0.1529,  ...,  0.0922, -0.1584,  0.2157]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

Using our custom backend, we can now see how TorchDynamo is able to handle
data-dependent control flow. Consider the function below, where the line
``if b.sum() < 0`` is the source of data-dependent control flow.


Используя наш собственный бэкэнд, мы теперь можем увидеть, как TorchDynamo работает с ситуацией, когда есть зависимость от данных. Рассмотрим:
``if b.sum() < 0``. 



In [37]:
def bar(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

opt_bar = torch.compile(bar, backend=custom_backend)
inp1 = torch.randn(10)
inp2 = torch.randn(10)
opt_bar(inp1, inp2)
opt_bar(inp1, -inp2)

custom backend called with FX graph:
opcode         name     target                                                  args              kwargs
-------------  -------  ------------------------------------------------------  ----------------  --------
placeholder    a        a                                                       ()                {}
placeholder    b        b                                                       ()                {}
call_function  abs_1    <built-in method abs of type object at 0x7fe6caa39880>  (a,)              {}
call_function  add      <built-in function add>                                 (abs_1, 1)        {}
call_function  truediv  <built-in function truediv>                             (a, add)          {}
call_method    sum_1    sum                                                     (b,)              {}
call_function  lt       <built-in function lt>                                  (sum_1, 0)        {}
output         output   output              

tensor([-0.0471, -0.1066,  0.6803, -0.0255, -0.3098, -0.0457, -0.0625,  0.1545,
         0.1780, -0.0358])

Мы видим что TorchDynamo разбил наш код на три части, которые соотвествуют:


1. ``x = a / (torch.abs(a) + 1)``
2. ``b = b * -1; return x * b``
3. ``return x * b``

Когда TorchDynamo встречает не поддерживаемые в функции, например зависящие от входных данных, он  разбивает код на части, дает передает все в стандатный Python интерпритатор, а потом возвращается к графу.

Давайте на примере разберемся, как TorchDynamo пройдет через ``<``.
Если ``b.sum() < 0``, то TorchDynamo запустит граф 1, даст
Python определить результат условного выражения, а затем запустит
граф 2. С другой стороны, если ``не b.sum() < 0``, то TorchDynamo
запустил бы граф 1, позволил Python определить результат условного выражения, затем
запустил график 3. <br>


Это подчеркивает основное различие между TorchDynamo и предыдущим  компиляторами для PyTorch.

Предыдущие решения либо упадут с ошибков, либо не правильно скомпилируются и ничего не произойдет.
TorchDynamo, с другой стороны, просто разобьет граф вычислений на части.

Что бы посмотреть как TorchDynamo разбивает граф на части мы можем вызвать: ``torch._dynamo.explain``



In [38]:
# Reset since we are using a different backend.
torch._dynamo.reset()
explanation, out_guards, graphs, ops_per_graph, break_reasons, explanation_verbose = torch._dynamo.explain(
    bar, torch.randn(10), torch.randn(10)
)
print(explanation_verbose)

Dynamo produced 2 graphs with 1 graph break and 5 ops
 Break reasons: 

1. generic_jump TensorVariable()
  File "/tmp/ipykernel_72586/3263660924.py", line 3, in bar
    if b.sum() < 0:
 
2. return_value
  File "/tmp/ipykernel_72586/3263660924.py", line 5, in <graph break in bar>
    return x * b
 
TorchDynamo compilation metrics:
Function                        Runtimes (s)
------------------------------  --------------
_compile                        0.0339, 0.0100
OutputGraph.call_user_compiler  0.0001, 0.0001


Чтобы максимизировать ускорение, разрывы графика должны быть ограничены.
Мы можем заставить TorchDynamo выдавать ошибку на первом разрыве графа ``fullgraph=True``:


In [39]:
opt_bar = torch.compile(bar, fullgraph=True)
try:
    opt_bar(torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_72586/3610564610.py", line 3, in <module>
    opt_bar(torch.randn(10), torch.randn(10))
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 337, in catch_errors
    return callback(frame, cache_size, hooks)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 104, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 262, in _convert_frame_assert
    return _compile(
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/utils.py", line 163, in time_wrapper
    r = func(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 324, in _compile
    out_code = transform_code_object(code, transform)
  File "/usr/local/lib/pyt

TorchDynamo не ломает граф модели, который мы использовали для анализа ускорения.

In [40]:
opt_model = torch.compile(init_model(), fullgraph=True)
print(opt_model(generate_data(16)[0]))

tensor([[ 0.0864,  0.3663,  0.0567,  ...,  0.1656, -0.3531, -0.0007],
        [ 0.1006,  0.2260, -0.0246,  ...,  0.2704, -0.3596, -0.1450],
        [-0.0218,  0.2373, -0.0612,  ...,  0.0124, -0.2159, -0.0417],
        ...,
        [ 0.0401,  0.3041, -0.0242,  ..., -0.0657, -0.2641,  0.0241],
        [ 0.0446,  0.4527, -0.0110,  ..., -0.1069, -0.2589, -0.0368],
        [-0.0208,  0.1708, -0.0923,  ...,  0.1494, -0.4422, -0.0778]],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>)


Наконец, если мы просто хотим, чтобы TorchDynamo выдал на torch FX граф,
мы можем использовать torch._dynamo.export. Обратите внимание, что ``torch._dynamo.export`` с
``fullgraph=True``, выдает ошибку, если TorchDynamo находит место разрывы графа.

In [118]:
try:
    torch._dynamo.export(bar, torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

model_exp = torch._dynamo.export(init_model(), generate_data(16)[0])
print(model_exp[0](generate_data(16)[0]))

Traceback (most recent call last):
  File "/tmp/ipykernel_5226/3120809583.py", line 2, in <module>
    torch._dynamo.export(bar, torch.randn(10), torch.randn(10))
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 601, in export
    result_traced = opt_f(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 209, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 337, in catch_errors
    return callback(frame, cache_size, hooks)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 104, in _fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 262, in _convert_frame_assert
    return _compile(
  File "/usr/local/lib/python3.10/site-packages/torch/_dynamo/utils.py", line 163, in time_wrapper
    r = func(*args, **kwargs)
  File "/usr/local/lib/py

tensor([[ 0.2901,  0.1556,  0.1009,  ..., -0.2593,  0.3228, -0.2279],
        [ 0.1345,  0.3276,  0.0695,  ..., -0.2303,  0.4229, -0.0485],
        [ 0.1782,  0.3309, -0.0861,  ..., -0.2063,  0.4099, -0.1587],
        ...,
        [ 0.2549,  0.2756, -0.0611,  ..., -0.1956,  0.2757, -0.1430],
        [ 0.1283,  0.2775, -0.0027,  ..., -0.1504,  0.2391, -0.1787],
        [ 0.1271,  0.2625, -0.1727,  ..., -0.0496,  0.4122, -0.1080]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
